In [28]:
from sagemaker import get_execution_role
import sagemaker as sage
import boto3
import time

In [22]:
role = get_execution_role()
sess = sage.Session()
sm = boto3.client('sagemaker')
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name

In [23]:
model = 'cntkdemo-ecr-demo-model' # just an arbitrary name for the model instantiation

In [24]:
# setup model artifact uri (see the "HowToPrepare" notebook)
model_data = 's3://sagemaker-{}-{}/output/cntkdemo-ecr-4-2018-01-21-13-26-49-604/output/model.tar.gz'.format(region,account)

In [25]:
# setup image reference name
image = '{}.dkr.ecr.{}.amazonaws.com/cntkdemo-ecr-4'.format(account, region) # image name on ECR

In [ ]:
%%time
create_model_response = sm.create_model(
    ModelName=model,
    ExecutionRoleArn=role,
    PrimaryContainer={
        'Image': image,
        'ModelDataUrl': model_data
    }
)

print(create_model_response['ModelArn'])

In [30]:
# configuring endpoint
cntkdemo_endpoint_config = 'cntkdemo-poc-endpoint-config-' + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())
cntkdemo_endpoint = 'cntkdemo-poc-endpoint-' + time.strftime("%Y%m%d%H%M", time.gmtime())

In [ ]:
%%time
create_endpoint_config_response = sm.create_endpoint_config(
    EndpointConfigName=cntkdemo_endpoint_config,
    ProductionVariants=[{
        'InstanceType': 'ml.m4.xlarge',
        'InitialInstanceCount': 1,
        'ModelName': model,
        'VariantName': 'cntkdemo'}])

print("Endpoint Config Arn: " + create_endpoint_config_response['EndpointConfigArn'])

In [ ]:
%%time
# be patience this block can require 5 or more minutes for the execution
create_endpoint_response = sm.create_endpoint(
    EndpointName=cntkdemo_endpoint,
    EndpointConfigName=cntkdemo_endpoint_config)

resp = sm.describe_endpoint(EndpointName=cntkdemo_endpoint)

status = resp['EndpointStatus']
print("Status: " + status)

sm.get_waiter('endpoint_in_service').wait(EndpointName=cntkdemo_endpoint)

resp = sm.describe_endpoint(EndpointName=cntkdemo_endpoint)

status = resp['EndpointStatus']
print("Status: " + status)

print("Arn: " + resp['EndpointArn'])

if status != 'InService':
    print ('Endpoint creation did not succeed')

In [35]:
print (cntkdemo_endpoint)

cntkdemo-poc-endpoint-201801240604
